Excellent question — and one that even many experienced Jupyter users ask.
Unfortunately, **Jupyter by itself doesn’t track where (which cell or line)** a variable was first created. But there are a few *practical workarounds* you can use to get that visibility 👇

---

### 🧭 **1. Track variable origins automatically (recommended trick)**

You can wrap your notebook’s global namespace in a custom tracker.

Add this **at the top of your notebook** (before imports):

```python
# --- Variable origin tracker ---
from IPython import get_ipython
import inspect, linecache

_created_vars = {}

def trace_exec(frame, event, arg):
    if event == "call": return trace_exec
    if event != "line": return
    code = frame.f_code
    line_no = frame.f_lineno
    cell_name = frame.f_globals.get('__name__', '')
    for var in frame.f_locals.keys():
        if var not in _created_vars:
            _created_vars[var] = f"{cell_name or '<cell>'}:{line_no}"
    return trace_exec

ip = get_ipython()
if ip:
    ip.events.register('pre_run_cell', lambda info: sys.settrace(trace_exec))
    ip.events.register('post_run_cell', lambda result: sys.settrace(None))
```

Now, whenever you execute a cell, it records *where each new variable first appeared*.

You can later check:

```python
_created_vars
```

✅ Output example:

```python
{
  'df': '<cell 5>:23',
  'num_cols': '<cell 7>:4',
  'cat_cols': '<cell 7>:5'
}
```

That means `df` was first defined on **line 23 of cell 5**, etc.

---

### 🧩 **2. Quick manual search for variable definitions**

You can use `grep` right from the notebook or terminal:

```bash
!grep -n "df =" 01_EDA.ipynb
```

Or, for any variable:

```bash
!grep -n "target_cols" 01_EDA.ipynb
```

That shows the **cell’s JSON line number** in the notebook file (less precise, but fast).

---

### 🔎 **3. IDE-based options**

If you open your notebook as a `.py` script (e.g. via “Export as Python script” or **Jupytext**):

* **VS Code / Windsurf / PyCharm** → Right-click → “Find All References” or “Go to Definition”
* You’ll see exactly which line the variable was created at.

---

### 🧠 Summary

| Method                  | What it shows                | Notes                         |
| ----------------------- | ---------------------------- | ----------------------------- |
| `%whos`                 | Current variables & memory   | Built-in, no line info        |
| `_created_vars` tracker | Cell + line of creation      | Add-on code, light overhead   |
| `grep "var ="`          | Approx cell in `.ipynb` JSON | Quick manual check            |
| Jupytext + IDE          | Full source line tracking    | Ideal for debugging notebooks |

---

Would you like me to give you a **ready-to-drop “Variable Origin Tracker” cell** that logs variable creation (cell, line, and timestamp) into a small `.csv` under your `/reports/debug/` folder? It’s handy for tracing variable lifecycles across your Telco notebooks.
